# <center> <span style="color:Blue"> **CAPSTONE PROJECT - The Battle of the Neighborhoods** </span> </center>

This notebook will be used for the capstone project. This is the final project of "**IBM Data Science Certificat Professionnel**" in partnership with Coursera

# Table of Content 

* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

# Introduction <a name="introduction"></a>

Our goal here is to provide insight on the different neighborhoods in Bordeaux Metropole that will help decider to choose thecorrect place to invest.   
Choosing a correct business emplacement will depend on several aspects: the type of business, the target, the population density, the competition, price per square meter of the local …).    
Choosing a correct place to live will depend also on several aspects : the age of the buyer, the family structure (single, couple, kids…) , their hobbies, the place of their work, commodities, transport services, price per square meter, type of housing facilities ….    
Finally, investors will be mainly interested in the capacity of the borrowers to pay of their loan, but they will also be interested in the potential price trends of the neighborhood in order to secure their investment.    

In order to achieve our goal of showing relationships between neighborhoods, a descriptive approach will be conduct. We will aggregate neighborhoods in clusters depending on the following information: 
* Real estate price 
* Most common type of real estate properties (apartments, houses…)
* Principal venues of the neighborhoods 


In [1]:
import pandas as pd 
import numpy as np
# import of request in order to use FOURSQUARE API 
import requests
import json
from pandas.io.json import json_normalize 
import folium

# Data <a name="data"></a>

First we need to import all the data necessary to conduct our project. We want to create clusters that allow us to decide which activities are most likely to succeed in a given neighborhood, we need to capture some informations like, the location of neighborhoods, the main venues of it, the real estate price, <span style="color:red">the population density</span> and the structure of real estate market. To acheive it, we will use several databases and API : 

A. We construct a dataset of towns and neighborhoods of Bordeaux Metropole    
B. We obtain real estate information from dvf API    
C. We enrich our data with FOURSQUARE venue   
D. We classify the neighborhoods   
E. <span style="color:red">We add population density (INSEE)</span>   

Our first objective is to obtain the coordinates of the different neighborhoods of Bordeaux Metropole.   
No dataset exists with the coordinates of neighborhoods, and the town to which their are attached. 
For this reason, we will have to decline our work in 4 parts : 
 * In a first time we found a dataset with all the neighborhoods of Bordeaux Metropole in a json format. This dataset does not contain the towns to witch neighborhoods are attached . The dataset is available here : 
https://www.data.gouv.fr/fr/datasets/quartiers-des-communes-sur-le-territoire-de-bordeaux-metropole/
 * In a second time, we will complete this dataset with a second one containing all the towns in the metropole (but without the neighborhoods) https://www.data.gouv.fr/fr/datasets/commune-sur-le-territoire-de-bordeaux-metropole/, 
 * Then, we will enrich our dataframe with the name of the neighborhoods' towns and postal codes using a restful API provided by the city of Bordeaux , 
 * Finally, we will use an geocoder API to complete our dataset with the coordinates of the additional towns.
Thanks to that, we will have a complete dataset of Bordeaux Metropole 

# A. Dataset of Bordeaux Metropole towns and neighborhoods
## 1. Bordeaux neighborhoods

| data type  | column name | content |
| ------------ | ------------- | ----------- |
|int  | GID | Primary key|
|Geo Point |  Geo point | Geopooints|
|Geo Shape |  Geo points  | geoshape of neighborhood|
|int  | GEOM_ERR | error code geopoints |
|String | NOM | neighborhood name|
|String | INSEE |Code INSEE of town |
|Boolean | VALIDE | validate neighborhood|
|Boolean | QUARPOLI | political neighborhood|
|Date | CDATE | creation date|
|Date | MDATE | modification date|

In [29]:
# we load the data  
with open('project_data/se_quart_s.json',encoding="UTF-8") as json_data:
    bordeaux_data = json.load(json_data)

Let's give a quick look at the data  

In [30]:
# we are interesed by the name of the neighborhood, 
#by the insee code (key to find town names) 
#and by the geo_point_2d (FOURSQUARE)
bordeaux_data['features'][0]['properties']

{'nom': 'Toctoucau',
 'insee': '33318',
 'mdate': '2016/12/02 10:36:56+01',
 'valide': '1',
 'quarpoli': '1',
 'geo_point_2d': [44.766930057, -0.733236943925],
 'cdate': '2016/03/31 19:45:28+02',
 'gid': 135}

In [31]:
#here we can take the geometry part to demarcate the neighborhood
bordeaux_data['features'][0]['geometry']

{'type': 'Polygon',
 'coordinates': [[[-0.7236434, 44.7806362],
   [-0.7263344, 44.7792995],
   [-0.7291259, 44.7779141],
   [-0.7307535, 44.7770885],
   [-0.7332663, 44.7758391],
   [-0.7338081, 44.7755676],
   [-0.7338882, 44.7755114],
   [-0.7339438, 44.7755508],
   [-0.7339448, 44.7755444],
   [-0.7339489, 44.7755483],
   [-0.7343875, 44.7753319],
   [-0.7403993, 44.7703313],
   [-0.7408029, 44.77],
   [-0.7441674, 44.7672699],
   [-0.7512356, 44.7615481],
   [-0.7517709, 44.761106],
   [-0.7522809, 44.7606862],
   [-0.752645, 44.7604251],
   [-0.7587161, 44.7555004],
   [-0.7531676, 44.7516568],
   [-0.7506216, 44.7498913],
   [-0.7501806, 44.7495772],
   [-0.7501583, 44.7495867],
   [-0.7461762, 44.7518437],
   [-0.744993, 44.7525111],
   [-0.7432294, 44.7535073],
   [-0.7415752, 44.7544483],
   [-0.7398473, 44.7554301],
   [-0.7383658, 44.7562666],
   [-0.7369807, 44.7570635],
   [-0.7358568, 44.7577103],
   [-0.7346792, 44.7583909],
   [-0.733496, 44.7590618],
   [-0.7334125, 4

In [32]:
# we create a data frame with the columns we are interested in :
#columns 
columns=['Neighborhood','INSEE','Latitude','Longitude','Geometry']
bordeaux_neighborhoods=pd.DataFrame(columns=columns)
bordeaux_neighborhoods

,Neighborhood,INSEE,Latitude,Longitude,Geometry


In [33]:
# loop in json file and fill dataframe a row at a time
for data in bordeaux_data['features']:
    neighborhood_name = data['properties']['nom']
    neighborhodd_insee = data['properties']['insee']
    neighborhood_latlon = data['properties']['geo_point_2d']
    neighborhood_lat = neighborhood_latlon[0]
    neighborhood_lon = neighborhood_latlon[1]
    neighborhodd_geometry=data['geometry']
    
    bordeaux_neighborhoods = bordeaux_neighborhoods.append({'Neighborhood': neighborhood_name,
                                                            'INSEE':neighborhodd_insee,
                                                            'Latitude': neighborhood_lat,
                                                            'Longitude': neighborhood_lon,
                                                            'Geometry':neighborhodd_geometry
                                                           }, ignore_index=True)

## 2. Bordeaux Towns

As this file do not have all towns of Bordeaux Metropole, we have found another dataset at : `"https://opendata.bordeaux-metropole.fr/explore/dataset/fv_commu_s/table/"` containing all bordeaux without the inforamtion about neighborhoods. 
We will add these towns and duplicate town name in neighborhood column.  

In [34]:
# we load the data containing all towns of bordeaux Metropole
with open('project_data/communes.json') as json_data_communes:
    bordeaux_data_comm = json.load(json_data_communes)

In [35]:
for data in bordeaux_data_comm: 
    insee= data['fields']['code_commune']
    #latitude= data['fields']['geo_point_2d'][0]
    #longitude=data['fields']['geo_point_2d'][1]
    #geometry =data['fields']['geo_shape']
    town= data['fields']['commune']
    if insee not in set(bordeaux_neighborhoods['INSEE']):
        bordeaux_neighborhoods= bordeaux_neighborhoods.append({'Neighborhood':town ,
                                                               'INSEE': insee
                                                               #'Latitude':latitude,
                                                               #'Longitude':longitude,
                                                               #'Geometry': geometry
                                                              },ignore_index=True)
                                                           

In [36]:
bordeaux_neighborhoods.head()

,Neighborhood,INSEE,Latitude,Longitude,Geometry
0,Toctoucau,33318,44.766930,-0.733237,"{'type': 'Polygon', 'coordinates': [[[-0.72364..."
1,3M-Bourgailh,33318,44.806158,-0.677987,"{'type': 'Polygon', 'coordinates': [[[-0.67910..."
2,Saige,33318,44.786531,-0.635364,"{'type': 'Polygon', 'coordinates': [[[-0.62840..."
3,Casino,33318,44.807801,-0.628748,"{'type': 'Polygon', 'coordinates': [[[-0.62803..."
4,Arlac,33281,44.822855,-0.626316,"{'type': 'Polygon', 'coordinates': [[[-0.62901..."


## 3. Completing missing towns and adding postal codes

Now that we have all the coordinates of each neighborhood, we just need to add the town to which they are attached. 
To do so, we will use an API provided by public opensoft data.

`url= 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=correspondance-code-insee-code-postal&facet=insee_com&facet=nom_dept&facet=nom_region&facet=statut&refine.insee_com=33063'`

In [37]:
for index, CODEINSEE in enumerate(bordeaux_neighborhoods['INSEE']): 
    url= f'https://public.opendatasoft.com/api/records/1.0/search/?dataset=correspondance-code-insee-code-postal&facet=insee_com&facet=nom_dept&facet=nom_region&facet=statut&refine.insee_com={CODEINSEE}'
    results=requests.get(url).json()
    bordeaux_neighborhoods.at[index,'postal_code']=results['records'][0]['fields']['postal_code']
    bordeaux_neighborhoods.at[index,'town']=results['records'][0]['fields']['nom_comm']
bordeaux_neighborhoods

,Neighborhood,INSEE,Latitude,Longitude,Geometry,postal_code,town
0,Toctoucau,33318,44.766930,-0.733237,"{'type': 'Polygon', 'coordinates': [[[-0.72364...",33600,PESSAC
1,3M-Bourgailh,33318,44.806158,-0.677987,"{'type': 'Polygon', 'coordinates': [[[-0.67910...",33600,PESSAC
2,Saige,33318,44.786531,-0.635364,"{'type': 'Polygon', 'coordinates': [[[-0.62840...",33600,PESSAC
3,Casino,33318,44.807801,-0.628748,"{'type': 'Polygon', 'coordinates': [[[-0.62803...",33600,PESSAC
4,Arlac,33281,44.822855,-0.626316,"{'type': 'Polygon', 'coordinates': [[[-0.62901...",33700,MERIGNAC
...,...,...,...,...,...,...,...
60,Saint-Jean-d'Illac,33422,NaN,NaN,NaN,33127,SAINT-JEAN-D'ILLAC
61,Bruges,33075,NaN,NaN,NaN,33520,BRUGES
62,Cubzac-les-Ponts,33143,NaN,NaN,NaN,33240,CUBZAC-LES-PONTS
63,Saint-Aubin-de-Médoc,33376,NaN,NaN,NaN,33160,SAINT-AUBIN-DE-MEDOC


In [82]:
# we load the data containing all towns of bordeaux Metropole -- https://france-geojson.gregoiredavid.fr/
with open('project_data/communes-33-gironde.geojson') as json_data_communes:
    bordeaux_data_comm2 = json.load(json_data_communes)

In [80]:
bordeaux_neighborhoods.drop(['geom_comm'],axis=1,inplace=True)

In [81]:
for index,town in enumerate(bordeaux_neighborhoods['INSEE']):
    try : 
        for data in bordeaux_data_comm2['features']: 
            if data['properties']['code'] == town : 
                bordeaux_neighborhoods.at[index,'Geometry']=data['geometry']
    except : 
        bordeaux_neighborhoods['Geometry']=''
bordeaux_neighborhoods

,Neighborhood,INSEE,Latitude,Longitude,Geometry,postal_code,town,neil_without_accent
0,Toctoucau,33318,44.766930,-0.733237,"{'type': 'Polygon', 'coordinates': [[[-0.64496...",33600,PESSAC,TOCTOUCAU
1,3M-Bourgailh,33318,44.806158,-0.677987,"{'type': 'Polygon', 'coordinates': [[[-0.64496...",33600,PESSAC,3M-BOURGAILH
2,Saige,33318,44.786531,-0.635364,"{'type': 'Polygon', 'coordinates': [[[-0.64496...",33600,PESSAC,SAIGE
3,Casino,33318,44.807801,-0.628748,"{'type': 'Polygon', 'coordinates': [[[-0.64496...",33600,PESSAC,CASINO
4,Arlac,33281,44.822855,-0.626316,"{'type': 'Polygon', 'coordinates': [[[-0.74630...",33700,MERIGNAC,ARLAC
...,...,...,...,...,...,...,...,...
60,Saint-Jean-d'Illac,33422,44.810186,-0.781526,"{'type': 'Polygon', 'coordinates': [[[-0.83708...",33127,SAINT-JEAN-D'ILLAC,SAINT-JEAN-D'ILLAC
61,Bruges,33075,44.884238,-0.611147,"{'type': 'Polygon', 'coordinates': [[[-0.62985...",33520,BRUGES,BRUGES
62,Cubzac-les-Ponts,33143,44.971119,-0.454723,"{'type': 'Polygon', 'coordinates': [[[-0.47272...",33240,CUBZAC-LES-PONTS,CUBZAC-LES-PONTS
63,Saint-Aubin-de-Médoc,33376,44.913422,-0.724673,"{'type': 'Polygon', 'coordinates': [[[-0.71408...",33160,SAINT-AUBIN-DE-MEDOC,SAINT-AUBIN-DE-MEDOC


Finaly, we can obtain missing coordinates by using the geocoder API. 
Before doing it, we will remove all special caracters from our neighborhoods.

In [38]:
#remove special caracters from neighborhoods column: 
import unicodedata 
for index,neil in enumerate(bordeaux_neighborhoods['Neighborhood']):
    bordeaux_neighborhoods.at[index,'neil_without_accent'] = ''.join((c for c in unicodedata.normalize('NFD', neil) if unicodedata.category(c) != 'Mn')).upper()

In [40]:
import geocoder

In [42]:
# initialize your variable to None
lat_lng_coords = None
for ind, lat,nei, code,town in zip(bordeaux_neighborhoods.index.values,bordeaux_neighborhoods['Latitude'],bordeaux_neighborhoods['neil_without_accent'],bordeaux_neighborhoods['postal_code'],bordeaux_neighborhoods['town']) : 
    #we merge all the geographical data used to find the coordinates : 
    if nei == town:
        geo_data = code + ','+town + ' CENTRE'
    else:
        geo_data=nei +','+ code +','+town
    #we focus our work only on missing coordinates :
    if np.isnan(lat) : 
# loop until we get the coordinates
        while(lat_lng_coords is None):
            g = geocoder.osm('{}, Gironde, France'.format(geo_data))
            lat_lng_coords = g.latlng
        bordeaux_neighborhoods.at[ind,'Latitude'] = lat_lng_coords[0]
        bordeaux_neighborhoods.at[ind,'Longitude'] = lat_lng_coords[1]
        lat_lng_coords = None

In [ ]:
#we save our dataframe in csv 
bordeaux_neighborhoods.to_csv('bordeaux_neighborhoods-enrichi2.csv',encoding="UTF-8")

In [ ]:
bordeaux_neighborhoods=pd.read_csv('bordeaux_neighborhoods-enrichi2.csv',index_col=0,encoding="UTF-8")
bordeaux_neighborhoods

In [16]:
len(bordeaux_neighborhoods['Neighborhood'].unique())

64

### Folium map
Let's plot our data on a folium map to see if it is accurate :

In [43]:
#coordinates of Bordeaux Metropole
import geocoder # import geocoder
g2 = geocoder.osm('Bordeaux, Gironde,France')
latlong= g2.latlng
latitude= latlong[0]
longitude = latlong[1]
print(f'latitude {latitude} & longitude {longitude}')

latitude 44.841225 & longitude -0.5800364


In [44]:
neighborhoods= folium.Map(location=[latitude,longitude],zoom_start=11)
for lat,long,bo,nei,geo in zip(bordeaux_neighborhoods['Latitude'],bordeaux_neighborhoods['Longitude'],bordeaux_neighborhoods['town'],bordeaux_neighborhoods['Neighborhood'],bordeaux_neighborhoods['Geometry']):
    label=(f'{bo}\n{nei}')
    label=folium.Popup(label, parse_html=True, encoding="latin_1")
    #pour ajouter les limites des quartiers : 
    #gj=folium.GeoJson(data=geo)
    #gj.add_to(neighborhoods)
    folium.CircleMarker(location=[lat,long],
                        popup=label,
                        radius=5,
                        fill=True,
                        fill_color='#3388ff',
                        fill_opacity=1,
                        parse_html=False).add_to(neighborhoods)
neighborhoods

# B. Real Estates 

It seems that our data set is ready. Now, we will add to the neighborhoods some information about real estates market

http://api.cquest.org/dvf?lat=48.85&lon=2.35&dist=200

Here again, let's give a look at the result returned from the API
The information we will keep is located in the `results['features']` section. 
In detail, we want to keep : 
* Value of the property: `['valeur_fonciere']`
* kind of property : `['type_local']`
* Area of the property :`['surface_relle_bati'`]
* Number of rooms : `['nombre_pieces_principales']`
* latitute :`['lat']`
* longitude: `['lon']`
* surface of parcel:  `['surface_terrain']`


In [88]:
# connect to foursquare and get the venues for ONLY TORONTO BOROUGH
RADIUS=500
temp=pd.DataFrame()
url_dvf='http://api.cquest.org/dvf?'
dvf_values = pd.DataFrame(columns=['town', 'INSEE','neighborhood', 'latitude', 'longitude','property','property_value','area', 'nb_rooms','parcel_surface'])
for  to, insee,nei,lat,long in zip(bordeaux_neighborhoods['town'],bordeaux_neighborhoods['INSEE'],bordeaux_neighborhoods['Neighborhood'],bordeaux_neighborhoods['Latitude'],bordeaux_neighborhoods['Longitude']):
    url=url_dvf+f'lat={round(lat,2)}&lon={round(long,2)}&dist={RADIUS}'
    results=requests.get(url).json()    
    for i in range(len(results['features'])):
        temp.at[i,'town']=to
        temp.at[i,'INSEE']=insee
        temp.at[i,'neighborhood']=nei
        temp.at[i,'latitude']=lat
        temp.at[i,'longitude']=long
        temp.at[i,'property']=str(results['features'][i]['properties'].get('type_local'))
        temp.at[i,'property_value']=results['features'][i]['properties'].get('valeur_fonciere')
        temp.at[i,'area']=results['features'][i]['properties'].get('surface_relle_bati')
        temp.at[i,'nb_rooms']=results['features'][i]['properties'].get('nombre_pieces_principales')
        temp.at[i,'parcel_surface']=results['features'][i]['properties'].get('surface_terrain')
    dvf_values=dvf_values.append(temp,ignore_index=True)
dvf_values.to_csv('dvf_values.csv')
dvf_values

,town,INSEE,neighborhood,latitude,longitude,property,property_value,area,nb_rooms,parcel_surface
0,PESSAC,33318,Toctoucau,44.766930,-0.733237,None,199000.0,NaN,NaN,1813.0
1,PESSAC,33318,Toctoucau,44.766930,-0.733237,Maison,199000.0,25.0,3.0,1177.0
2,PESSAC,33318,Toctoucau,44.766930,-0.733237,None,925300.0,NaN,NaN,614.0
3,PESSAC,33318,Toctoucau,44.766930,-0.733237,None,925300.0,NaN,NaN,1033.0
4,PESSAC,33318,Toctoucau,44.766930,-0.733237,None,867010.0,NaN,NaN,1033.0
...,...,...,...,...,...,...,...,...,...,...
163899,BORDEAUX,33063,Centre ville,44.842351,-0.583334,Appartement,447000.0,49.0,2.0,66.0
163900,BORDEAUX,33063,Centre ville,44.842351,-0.583334,Appartement,447000.0,48.0,2.0,66.0
163901,BORDEAUX,33063,Centre ville,44.842351,-0.583334,Appartement,630000.0,49.0,2.0,66.0
163902,BORDEAUX,33063,Centre ville,44.842351,-0.583334,Local industriel. commercial ou assimilé,630000.0,107.0,0.0,66.0


In [89]:
#we clean the results by droping the row with missing values 
dvf_values_clean=dvf_values
dvf_values_clean['area']=dvf_values_clean['area'].replace(0.0,np.nan)
dvf_values_clean=dvf_values_clean.dropna(subset=['area']).reset_index(drop=True)

In [90]:
dvf_values_clean.shape

(86132, 10)

In [97]:
# we group by town & neighborhood, we apply mean value on numerical values and we extract most common value from string column( property type)
dvf_values_grouped=dvf_values_clean.groupby(['town','INSEE','neighborhood','latitude','longitude'],as_index=False).agg({
    'property':lambda x: x.value_counts().index[0],
    'property_value':'mean',
    'area':'mean',
    'nb_rooms':'mean',
    'parcel_surface':'mean'})

In [98]:
dvf_values_grouped

,town,INSEE,neighborhood,latitude,longitude,property,property_value,area,nb_rooms,parcel_surface
0,AMBARES-ET-LAGRAVE,33003,Ambarès-et-Lagrave,44.925901,-0.488185,Maison,196837.323739,80.982609,3.886957,627.771739
1,AMBES,33004,Ambès,45.012156,-0.529185,Maison,202769.247059,99.235294,3.988235,562.176471
2,ARTIGUES-PRES-BORDEAUX,33013,Artigues-près-Bordeaux,44.859925,-0.492353,Maison,307416.372340,89.368421,3.557895,1012.290323
3,BASSENS,33032,Bassens,44.901497,-0.516324,Maison,276743.657737,75.795620,3.474453,432.339450
4,BEGLES,33039,Bègles,44.807456,-0.548854,Appartement,643943.360267,79.535421,2.500000,714.700472
...,...,...,...,...,...,...,...,...,...,...
57,SAINT-LOUIS-DE-MONTFERRAND,33434,Saint-Louis-de-Montferrand,44.953670,-0.533967,Maison,161731.343750,78.187500,3.000000,846.136364
58,SAINT-MEDARD-EN-JALLES,33449,Saint-Médard-en-Jalles,44.895889,-0.717076,Maison,282951.248062,86.240310,3.031008,598.760870
59,SAINT-VINCENT-DE-PAUL,33487,Saint-Vincent-de-Paul,44.954980,-0.469296,Maison,202302.941176,102.647059,4.470588,680.823529
60,TALENCE,33522,Talence,44.808844,-0.587963,Appartement,292644.060966,66.905660,2.518685,428.645274


In [99]:
len(dvf_values_grouped['neighborhood'].unique())

61

In [100]:
bordeaux_neighborhoods[bordeaux_neighborhoods['Neighborhood']=='3M-Bourgailh']

,Neighborhood,INSEE,Latitude,Longitude,Geometry,postal_code,town,neil_without_accent
1,3M-Bourgailh,33318,44.806158,-0.677987,"{'type': 'Polygon', 'coordinates': [[[-0.64496...",33600,PESSAC,3M-BOURGAILH


In [114]:
world_geo['features'][0]['properties']['code']

{'code': '33200', 'nom': 'Haillan'}

In [118]:
with open('project_data/communes-33-gironde.geojson',encoding="UTF-8") as json_data:
    world_geo = json.load(json_data)

# create a plain world map
world_map = folium.Map(location=[0, 0], zoom_start=2, tiles='Mapbox Bright')
# generate choropleth map using the total immigration of each country to Canada from 1980 to 2013
world_map.choropleth(
    geo_data=world_geo,
    data=dvf_values_grouped,
    columns=['INSEE', 'property_value'],
    key_on='feature.properties.code',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2
)

# display map
world_map